# Importation des Bibliothèques et Préparation des Données

In [15]:
import nltk
import os
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Télécharger les ressources nécessaires de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Chemin vers le dossier contenant les documents
corpus_folder = "./corpus_covid19"


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chalh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chalh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chalh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Traitement du Texte et Construction du Corpus

In [16]:
# Liste des noms de fichiers dans le dossier
file_names = os.listdir(corpus_folder)

# Initialiser une liste pour contenir les mots de chaque document
documents_words = []

# Initialiser le lemmatiseur
lemmatizer = WordNetLemmatizer()

# Parcourir chaque fichier, lire son contenu et segmenter en mots
for file_name in file_names:
    file_path = os.path.join(corpus_folder, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        words = word_tokenize(content)  # Segmenter en mots
        
        # Filtrer les mots vides de la langue anglaise
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word.lower() not in stop_words]
        
        # Lemmatisation des mots
        words = [lemmatizer.lemmatize(word) for word in words]
        
        # Supprimer les caractères de ponctuation
        words = [word for word in words if re.match('^[a-zA-Z0-9]+$', word)]
        
        documents_words.append(words)


# Construction de la Matrice d'Incidence

In [17]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Initialiser une liste pour contenir le texte de chaque document
documents_text = []

# Lemmatisation des mots dans chaque document
lemmatized_documents = []
for document in documents_words:
    lemmatized_documents.append(" ".join(document))

# Initialiser le vectoriseur pour la construction de la matrice d'incidence
vectorizer = CountVectorizer()

# Construire la matrice d'incidence documents-termes
matrix_documents_terms = vectorizer.fit_transform(lemmatized_documents)

# Obtenez le dictionnaire des termes avec leurs indices associés
terms_dictionary = vectorizer.get_feature_names_out()


# Création de l'Index Inversé



In [18]:
from collections import defaultdict

# Création de l'index inversé à partir de la matrice d'incidence documents-termes
def create_inverted_index(matrix_documents_terms, terms_dictionary):
    # Initialiser un dictionnaire par défaut où chaque clé contiendra une liste vide
    inverted_index = defaultdict(list)
    
    # Obtenir la matrice d'incidence sous forme d'un tableau numpy
    incidence_matrix = matrix_documents_terms.toarray()
    
    # Parcourir chaque terme dans le dictionnaire des termes
    for term_index, term in enumerate(terms_dictionary):
        # Trouver les indices des documents où le terme apparaît
        document_indices = np.where(incidence_matrix[:, term_index] > 0)[0]
        # Ajouter ces indices à la liste du terme dans l'index inversé
        inverted_index[term] = document_indices.tolist()
    
    return inverted_index

# Utiliser la matrice d'incidence et le dictionnaire des termes pour créer l'index inversé
inverted_index = create_inverted_index(matrix_documents_terms, terms_dictionary)

# Afficher l'index inversé
for term, doc_indices in inverted_index.items():
    print(f"Terme '{term}': {doc_indices}")


Terme '08': [9]
Terme '082': [9]
Terme '10': [1, 2, 5, 9]
Terme '100': [1, 2, 9]
Terme '1000': [4]
Terme '101': [3, 5, 9]
Terme '1016': [9]
Terme '103': [9]
Terme '1038': [9]
Terme '104': [4]
Terme '105': [2, 4]
Terme '106': [4, 9]
Terme '107': [4, 5, 9]
Terme '109': [9]
Terme '11': [1, 2, 3, 4, 5, 9]
Terme '112': [3, 9]
Terme '113': [4]
Terme '115': [3, 9]
Terme '1153': [9]
Terme '116': [9]
Terme '1172': [9]
Terme '12': [1, 2, 5, 8, 9]
Terme '120': [9]
Terme '121': [9]
Terme '122': [9]
Terme '123': [4, 8]
Terme '126': [9]
Terme '1265': [9]
Terme '127': [3]
Terme '128': [3, 9]
Terme '129': [9]
Terme '12th': [1]
Terme '13': [0, 1, 2, 3, 4, 9]
Terme '132': [9]
Terme '1339': [9]
Terme '135': [2]
Terme '1378': [9]
Terme '139': [8, 9]
Terme '1393': [9]
Terme '14': [1, 3, 4, 6, 9]
Terme '1421': [2]
Terme '143': [3, 5]
Terme '148': [8]
Terme '15': [1, 2, 3, 4, 5, 9]
Terme '150': [5]
Terme '153': [9]
Terme '16': [1, 2, 3, 4, 9]
Terme '160': [2]
Terme '1617': [9]
Terme '163': [9]
Terme '164': [

# Recherche Simple et Normalisée

In [19]:
def simple_search(query, inverted_index, matrix_documents_terms):
    # Rechercher l'index inversé pour les documents contenant le terme de la requête
    if query in inverted_index:
        doc_indices = inverted_index[query]
        # Compter les occurrences du terme de la requête dans chaque document
        term_counts = matrix_documents_terms.toarray()[:, terms_dictionary.tolist().index(query)]
        # Associer chaque document à son nombre d'occurrences du terme
        doc_occurrences = [(index, term_counts[index]) for index in doc_indices]
        # Trier les documents par le nombre d'occurrences décroissant
        sorted_docs = sorted(doc_occurrences, key=lambda x: x[1], reverse=True)
        return [doc[0] for doc in sorted_docs]
    else:
        return []

def normalized_search(query, inverted_index, matrix_documents_terms):
    # Rechercher l'index inversé pour les documents contenant le terme de la requête
    if query in inverted_index:
        doc_indices = inverted_index[query]
        # Compter les occurrences du terme de la requête dans chaque document
        term_counts = matrix_documents_terms.toarray()[:, terms_dictionary.tolist().index(query)]
        # Normaliser les comptages par la longueur totale des documents
        doc_lengths = matrix_documents_terms.toarray().sum(axis=1)
        # Éviter la division par zéro
        doc_lengths[doc_lengths == 0] = 1
        normalized_counts = term_counts / doc_lengths
        # Associer chaque document à son nombre normalisé d'occurrences du terme
        doc_occurrences = [(index, normalized_counts[index]) for index in doc_indices]
        # Trier les documents par le score normalisé décroissant
        sorted_docs = sorted(doc_occurrences, key=lambda x: x[1], reverse=True)
        return [doc[0] for doc in sorted_docs]
    else:
        return []

# Exemple d'utilisation de la fonction simple_search
simple_results = simple_search('covid', inverted_index, matrix_documents_terms)
print("Résultats de la recherche simple:", simple_results)

# Exemple d'utilisation de la fonction normalized_search
normalized_results = normalized_search('covid', inverted_index, matrix_documents_terms)
print("Résultats de la recherche normalisée:", normalized_results)


Résultats de la recherche simple: [0, 8]
Résultats de la recherche normalisée: [0, 8]


# Recherche TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialiser le vectoriseur TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Construire la matrice TF-IDF pour le corpus lemmatisé
tfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_documents)

# Fonction pour effectuer une recherche TF-IDF
def tfidf_search(query, tfidf_vectorizer, tfidf_matrix):
    # Transformer la requête en vecteur TF-IDF
    query_vector = tfidf_vectorizer.transform([query])
    # Calculer la similarité cosinus entre la requête et les documents du corpus
    cosine_similarities = (tfidf_matrix * query_vector.T).toarray().flatten()
    # Trier les documents par similarité cosinus décroissante
    sorted_doc_indices = np.argsort(-cosine_similarities)
    return sorted_doc_indices, cosine_similarities[sorted_doc_indices]

# Exemple d'utilisation de la fonction tfidf_search
query = 'plasma  '
search_results, similarities = tfidf_search(query, tfidf_vectorizer, tfidf_matrix)
print("Résultats de la recherche TF-IDF:", search_results)
print("Scores de similarité:", similarities)


Résultats de la recherche TF-IDF: [2 4 9 5 0 1 3 6 7 8]
Scores de similarité: [0.40838684 0.01181393 0.00775966 0.0074975  0.         0.
 0.         0.         0.         0.        ]


 # Requêtes Booléennes

In [22]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Prétraitement de la requête
def preprocess_query(query):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))  # Assurez-vous que la langue des stopwords correspond à celle de vos documents
    words = word_tokenize(query)
    return [lemmatizer.lemmatize(w.lower()) for w in words if w.isalpha() and w not in stop_words]

# Fonction de recherche booléenne
def boolean_search(inverted_index, query):
    query_tokens = preprocess_query(query)
    stack = []

    for token in query_tokens:
        if token.lower() in inverted_index:
            stack.append(set(inverted_index[token.lower()]))
        elif token.upper() in ['AND', 'OR', 'NOT'] and len(stack) >= 2:
            right = stack.pop()
            left = stack.pop()
            if token.upper() == 'AND':
                stack.append(left & right)
            elif token.upper() == 'OR':
                stack.append(left | right)
        elif token.upper() == 'NOT' and stack:
            operand = stack.pop()
            universe = set(range(len(documents_text)))  # Assurez-vous que c'est la bonne référence pour la taille de votre corpus
            stack.append(universe - operand)

    if stack:
        return sorted(stack[-1])
    else:
        return []


# Effectuer les requêtes booléennes
queries = [
    'desease AND severe',
    'antibody AND plasma AND (cells OR receptors)',
    'antimalarial drugs OR antiviral agents OR immunomodulators',
    'NOT plasma AND risk of infection AND NOT restrictions',
    '(older adults AND antibodies) AND (genomes OR variant)'
]

# Appliquer les recherches booléennes pour chaque requête
for i, q in enumerate(queries, start=1):
    result = boolean_search(inverted_index, q)
    print(f'Requête booléenne {i}: {result}')


Requête booléenne 1: [1, 2, 3, 4, 5, 9]
Requête booléenne 2: [3, 5, 9]
Requête booléenne 3: [1, 2, 3, 4, 5, 9]
Requête booléenne 4: [8, 9]
Requête booléenne 5: [1, 5]


# Requêtes Textuelles Complexes

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialiser le vectoriseur TF-IDF avec les documents prétraités
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_documents)

# Fonction pour exécuter les requêtes complexes et trouver les documents pertinents
def complex_textual_search(query, tfidf_vectorizer, tfidf_matrix):
    # Transformer la requête en vecteur TF-IDF
    query_vector = tfidf_vectorizer.transform([query])
    # Calculer la similarité cosinus entre la requête transformée et les documents
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()
    # Trier les indices des documents par similarité décroissante
    relevant_docs_indices = cosine_sim.argsort()[::-1]
    return relevant_docs_indices, cosine_sim[relevant_docs_indices]

# Requêtes complexes
complex_queries = [
    "antibody treatments",
    "efficacy and safety of the treatments",
    "family access to hospitals",
    "contact tracing results",
    "genomic analysis of SARS-CoV-2 disease"
]

# Exécuter et afficher les résultats pour chaque requête complexe
for i, query in enumerate(complex_queries, start=1):
    indices, scores = complex_textual_search(query, tfidf_vectorizer, tfidf_matrix)
    print(f"Requête textuelle complexe {i}: {query}")
    print(f"Indices des documents les plus pertinents: {indices[:5]}")
    print(f"Scores de similarité: {scores[:5]}\n")

Requête textuelle complexe 1: antibody treatments
Indices des documents les plus pertinents: [4 2 9 1 5]
Scores de similarité: [0.08392587 0.04566077 0.04469551 0.02522231 0.02303223]

Requête textuelle complexe 2: efficacy and safety of the treatments
Indices des documents les plus pertinents: [3 4 5 2 6]
Scores de similarité: [0.05777396 0.04916151 0.03119947 0.02657668 0.00916165]

Requête textuelle complexe 3: family access to hospitals
Indices des documents les plus pertinents: [8 7 0 2 4]
Scores de similarité: [0.23457488 0.03927397 0.02859546 0.02786942 0.00733859]

Requête textuelle complexe 4: contact tracing results
Indices des documents les plus pertinents: [3 4 5 2 9]
Scores de similarité: [0.01658094 0.01354269 0.01031355 0.00936294 0.00177903]

Requête textuelle complexe 5: genomic analysis of SARS-CoV-2 disease
Indices des documents les plus pertinents: [4 5 2 9 3]
Scores de similarité: [0.0827054  0.07503227 0.07227719 0.05995263 0.04200599]
